In [10]:
import kaleido

In [1]:
import yfinance as yf

df = yf.download('FCL.NS')
df['200_ma'] = df['Close'].rolling(200).mean()

[*********************100%%**********************]  1 of 1 completed


In [3]:
# The code above is actually downloading Tesla daily price data, and then calculating the 200 day simple moving average

In [4]:
df

,Open,High,Low,Close,Adj Close,Volume,200_ma
Date,,,,,,,
2015-01-15,23.995001,24.985001,23.000000,24.605000,23.727531,631430,NaN
2015-01-16,24.400000,24.400000,22.610001,23.600000,22.758373,41940,NaN
2015-01-19,24.100000,25.100000,24.100000,24.934999,24.045769,331430,NaN
2015-01-20,25.000000,25.000000,24.295000,24.295000,23.428589,1200,NaN
2015-01-21,23.975000,24.695000,23.730000,24.424999,23.553955,950,NaN
...,...,...,...,...,...,...,...
2024-02-15,410.950012,429.000000,398.799988,407.399994,407.399994,3767917,314.38500
2024-02-16,412.500000,449.950012,410.250000,438.250000,438.250000,3790159,315.38225
2024-02-19,458.850006,458.850006,432.350006,436.149994,436.149994,1611909,316.36650


In [23]:
import numpy as np
import pandas as pd
import yfinance as yf

TICKER = 'HDFCBANK.NS'


def best_fit_slope(y: np.array) -> float:
    '''
    Determine the slope for the linear regression line

    Parameters
    ----------
    y : TYPE
        The time-series to find the linear regression line for

    Returns
    -------
    m : float
        The gradient (slope) of the linear regression line
    '''
    
    x = np.arange(0, y.shape[0])
    
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    
    return np.sum((x-x_bar)*(y-y_bar))/np.sum((x-x_bar)**2)


def apply_trend_template(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Apply Mark Minervini's trend criteria and obtain a new boolean column
    to indicate where the criteria is applied
    
    Parameters
    ----------
    df : pd.DataFrame
        The stock price dataframe
        
    Returns
    -------
    df : pd.DataFrame
        The stock price dataframe with the new trend boolean column
    '''
    
    # Find the moving averages
    df['200_ma'] = df['Close'].rolling(200).mean()
    df['150_ma'] = df['Close'].rolling(150).mean()
    df['50_ma'] = df['Close'].rolling(50).mean()
    
    # Determine the 52 week high and low
    df['52_week_low'] = df['Close'].rolling(52*5).min()
    df['52_week_high'] = df['Close'].rolling(52*5).max()
    
    # Get the linear regression slope of the 200 day SMA
    df['slope'] = df['200_ma'].rolling(40).apply(best_fit_slope)
    
    # Constraints for the trend template
    df['trend_template'] = (
        (df['Close'] > df['200_ma'])
        & (df['Close'] > df['150_ma'])
        & (df['150_ma'] > df['200_ma'])
        & (df['slope'] > 0)
        & (df['50_ma'] > df['150_ma'])
        & (df['50_ma'] > df['200_ma'])
        & (df['Close'] > df['50_ma'])
        & (df['Close']/df['52_week_low'] > 1.3)
        & (df['Close']/df['52_week_high'] > 0.8) 
    )
    
    return df

if __name__ == '__main__':

    df = yf.download(f'{TICKER}').reset_index()
    df = apply_trend_template(df)
    
    if df['trend_template'].values[-1]:
        print(f'{TICKER} is trending!')
    else:
        print(f'{TICKER} is not trending :(')

[*********************100%%**********************]  1 of 1 completed


HDFCBANK.NS is not trending :(


In [16]:
tickers = ['FCL.NS', 'IRB.NS', 'GPPL.NS', 'HDFCBANK.NS']

for ticker in tickers:
    df = yf.download(f'{TICKER}').reset_index()
    df = apply_trend_template(df)
    
    if df['trend_template'].values[-1]:
        print(f'{TICKER} is trending!')
    else:
        print(f'{TICKER} is not trending :(')

[*********************100%%**********************]  1 of 1 completed


FCL.NS is trending!


[*********************100%%**********************]  1 of 1 completed


FCL.NS is trending!


[*********************100%%**********************]  1 of 1 completed


FCL.NS is trending!


[*********************100%%**********************]  1 of 1 completed


FCL.NS is trending!


In [24]:
from datetime import datetime, timedelta

TICKERS = ['FCL.NS', 'IRB.NS', 'GPPL.NS', 'HDFCBANK.NS','WIPRO.NS']

data = yf.download(
    tickers=TICKERS,
    start=datetime.now() - timedelta(weeks=55),
    threads=True,
    group_by='ticker',
)

data = data.T

trending_tickers = []

for ticker in TICKERS:
    df = apply_trend_template(data.loc[ticker, :].T)

    if df['trend_template'].values[-1]:
        trending_tickers.append(ticker)

if len(trending_tickers) > 0:
    print(trending_tickers)
else:
    print('No tickers are trending :(')


[*********************100%%**********************]  5 of 5 completed

['FCL.NS', 'IRB.NS', 'GPPL.NS', 'WIPRO.NS']
